In [ ]:
from enum import Enum

# Settings
BET_AMOUNT = 10
RAISE_AMOUNT = 20
STARTING_POT = 60

class Action(Enum):
    """
    Each player can check(x), bet(b), fold(f), call(c), and raise(r).
    The bet is 10 chips.
    The raise is set to 20 chips and there are no reraises allowed.
    """
    CHECK = ('x', 0)
    CALL = ('c', None)
    FOLD = ('f', 0)
    BET = ('b', BET_AMOUNT)
    RAISE = ('r', RAISE_AMOUNT)

    def __init__(self, code, default_amount):
        self.code = code
        self.default_amount = default_amount

    def __repr__(self):
        # For BET and RAISE, include the default amount in the representation (e.g., "b10" or "r20").
        if self in (Action.BET, Action.RAISE):
            return f"{self.code}{self.default_amount}"
        else:
            return self.code

In [ ]:
class Player:
    def __init__(self, name: str):
        self.name = name

class BettingRoundState:
    """
    Represents the state of a betting round (e.g., the flop round) in Texas Hold'em.
    It includes:
      - The list of players participating in the round.
      - The pot amount.
      - The current bet that players must match.
      - Whether a raise has been made.
      - The contributions of each player in the round.
      - The set of players who have folded.
    """
    def __init__(self, players):
        self.players = players
        self.pot = STARTING_POT
        self.current_bet = 0
        self.raise_done = False
        self.contributions = {i: 0 for i in range(len(players))}
        self.folded = set()

    def clone(self):
        new_state = BettingRoundState(self.players)
        new_state.pot = self.pot
        new_state.current_bet = self.current_bet
        new_state.raise_done = self.raise_done
        new_state.contributions = self.contributions.copy()
        new_state.folded = self.folded.copy()

        return new_state

    def get_active_players(self):
        return [i for i in range(len(self.players)) if i not in self.folded]

    def get_allowed_actions(self, player_index):
        """
        Determines the allowed actions for the given player based on the current state:
          - If no bet has been placed: allow CHECK, BET, and FOLD.
          - If a bet exists: allow CALL and FOLD, and if no raise has occurred, also allow RAISE.
        """
        if self.current_bet == 0:
            return [Action.FOLD, Action.CHECK, Action.BET]
        else:
            amount_to_call = self.current_bet - self.contributions[player_index]
            if amount_to_call <= 0:
                return []
            actions = [Action.FOLD, Action.CALL]
            if not self.raise_done:
                actions.append(Action.RAISE)

            return actions

    def apply_action(self, player_index, action, last_aggressor_index, players_seen):
        """
        Updates the game state based on the specified action taken by the player, and returns:
          - new_state: the updated state,
          - new_last_aggressor_index: the updated index of the last player who bet or raised,
          - new_players_seen: the updated set of active players who have acted since the last bet.
        """
        new_state = self.clone()
        new_last_aggressor_index = last_aggressor_index
        new_players_seen = players_seen.copy()

        if action == Action.FOLD:
            new_state.folded.add(player_index)
        elif action == Action.CHECK:
            pass
        elif action == Action.BET:
            amount = Action.BET.default_amount
            new_state.current_bet = amount
            new_state.pot += amount
            new_state.contributions[player_index] += amount
            new_last_aggressor_index = player_index
            new_state.raise_done = False
            new_players_seen.clear()
        elif action == Action.CALL:
            call_amount = new_state.current_bet - new_state.contributions[player_index]
            new_state.pot += call_amount
            new_state.contributions[player_index] += call_amount
        elif action == Action.RAISE:
            raise_amount = Action.RAISE.default_amount
            additional = raise_amount - new_state.contributions[player_index]
            new_state.pot += additional
            new_state.contributions[player_index] = raise_amount #Corrected Line
            new_state.current_bet = raise_amount
            new_last_aggressor_index = player_index
            new_state.raise_done = True
            new_players_seen.clear()

        # In a no-bet state, if the action is CHECK or FOLD, record that this player has acted.
        if self.current_bet == 0 and (action == Action.FOLD or action == Action.CHECK):
            new_players_seen.add(player_index)

        return new_state, new_last_aggressor_index, new_players_seen

    def get_next_player(self, current_index):
        next_player = (current_index + 1) % len(self.players)
        while next_player in self.folded and len(self.get_active_players()) > 1:
            next_player = (next_player + 1) % len(self.players)

        return next_player

    def is_round_over(self, players_seen, last_aggressor_index=None, next_player=None):
        """
        Determines whether the betting round is over:
          1. If only one player remains (not folded), the round is over.
          2. If there is no bet and all active players have acted, the round is over.
          3. If a bet exists and the next player is the last aggressor (i.e., the one who initiated the bet/raise),
            the round is over.
        Note: In a bet state, last_aggressor_index and next_player must be provided.
        """
        active_players = self.get_active_players()
        if len(active_players) == 1:
            return True

        if self.current_bet == 0:
            return set(active_players).issubset(players_seen)

        return last_aggressor_index is not None and next_player == last_aggressor_index

In [ ]:
def simulate_turn(player_index, last_aggressor_index, players_seen, action_sequence, state, results):
    # if only one active player, round is over
    active_players = state.get_active_players()
    if len(active_players) == 1:
        results.append((action_sequence.copy(), state.pot))
        return

    allowed_actions = state.get_allowed_actions(player_index)
    if not allowed_actions:
        next_player = state.get_next_player(player_index)
        simulate_turn(next_player, last_aggressor_index, players_seen, action_sequence, state, results)
        return

    for action in allowed_actions:
        new_state, new_last_aggressor_index, new_players_seen = state.apply_action(
            player_index, action, last_aggressor_index, players_seen
        )

        new_sequence = action_sequence.copy()
        new_sequence.append(repr(action))
        next_player = new_state.get_next_player(player_index)

        if new_state.is_round_over(new_players_seen, new_last_aggressor_index, next_player):
            results.append((new_sequence, new_state.pot))
        else:
            simulate_turn(next_player, new_last_aggressor_index, new_players_seen, new_sequence, new_state, results)

In [ ]:
def main():
    players = [Player("P1"), Player("P2"), Player("P3")]
    initial_state = BettingRoundState(players)
    results = []

    simulate_turn(0, None, set(), [], initial_state, results)

    seen = set()
    unique_results = []

    for seq, pot in results:
        seq_str = ":".join(seq)
        if seq_str not in seen:
            seen.add(seq_str)
            unique_results.append((seq_str, pot))

    print(f"Total possible unique outcomes: {len(unique_results)}\n")
    for seq_str, pot in sorted(unique_results, reverse=True):
        print(f"{seq_str}, pot={pot}")

In [ ]:
if __name__ == "__main__":
    main()

Total possible unique outcomes: 61

x:x:x, pot=60
x:x:f, pot=60
x:x:b10:r20:f:f, pot=90
x:x:b10:r20:f:c, pot=100
x:x:b10:r20:c:f, pot=110
x:x:b10:r20:c:c, pot=120
x:x:b10:f:r20:f, pot=90
x:x:b10:f:r20:c, pot=100
x:x:b10:f:f, pot=70
x:x:b10:f:c, pot=80
x:x:b10:c:r20:f:f, pot=100
x:x:b10:c:r20:f:c, pot=110
x:x:b10:c:r20:c:f, pot=110
x:x:b10:c:r20:c:c, pot=120
x:x:b10:c:f, pot=80
x:x:b10:c:c, pot=90
x:f:x, pot=60
x:f:f, pot=60
x:f:b10:r20:f, pot=90
x:f:b10:r20:c, pot=100
x:f:b10:f, pot=70
x:f:b10:c, pot=80
x:b10:r20:f:f, pot=90
x:b10:r20:f:c, pot=100
x:b10:r20:c:f, pot=110
x:b10:r20:c:c, pot=120
x:b10:f:r20:f, pot=90
x:b10:f:r20:c, pot=100
x:b10:f:f, pot=70
x:b10:f:c, pot=80
x:b10:c:r20:f:f, pot=100
x:b10:c:r20:f:c, pot=110
x:b10:c:r20:c:f, pot=110
x:b10:c:r20:c:c, pot=120
x:b10:c:f, pot=80
x:b10:c:c, pot=90
f:x:x, pot=60
f:x:f, pot=60
f:x:b10:r20:f, pot=90
f:x:b10:r20:c, pot=100
f:x:b10:f, pot=70
f:x:b10:c, pot=80
f:f, pot=60
f:b10:r20:f, pot=90
f:b10:r20:c, pot=100
f:b10:f, pot=70
f:b10